# YOLO Object Detection - Light Inference

This notebook demonstrates optimized, lightweight object detection using YOLO on edge devices.
Optimized for devices with limited resources (2GB+ RAM, CPU-only support).

## Device Compatibility
- Compatible with Raspberry Pi
- Compatible with Jetson Nano
- Compatible with Jetson AGX Orin (but full-inference version recommended)

## Optimizations
- Uses quantized models
- Reduced input resolution
- Batch size of 1
- Memory-efficient processing

In [ ]:
import sys
sys.path.append('../..')
from base_config import DeviceConfig
from monitoring import DeviceMonitor

# Initialize device configuration
config = DeviceConfig()
monitor = DeviceMonitor()

# Check device compatibility
requirements = {
    "gpu_required": False,
    "min_memory": "2GB"
}

if not config.check_notebook_compatibility(requirements):
    raise RuntimeError("Device does not meet minimum requirements for this notebook")

print(f"Device {config.device_type} is compatible with this notebook")
print("Available features:", config.get_device_features())


In [ ]:
# Import required libraries
import cv2
import numpy as np
from supervision import BoxAnnotator

# Import appropriate inference backend based on device
if config.device_type == 'raspberry-pi':
    import tflite_runtime.interpreter as tflite
    # Load quantized TFLite model
    interpreter = tflite.Interpreter(model_path='models/yolov8n_quantized.tflite')
    interpreter.allocate_tensors()
else:
    from ultralytics import YOLO
    # Load optimized PyTorch model
    model = YOLO('yolov8n.pt')

# Set optimized parameters
INPUT_SIZE = (320, 320)  # Reduced resolution
CONF_THRESHOLD = 0.25    # Adjusted confidence threshold


In [ ]:
def preprocess_image(frame):
    """Optimize image preprocessing for memory efficiency."""
    # Resize with reduced memory footprint
    frame = cv2.resize(frame, INPUT_SIZE, interpolation=cv2.INTER_AREA)
    
    # Normalize using integer arithmetic when possible
    frame = frame.astype(np.float32) / 255.0
    return frame

def process_frame(frame):
    """Memory-efficient frame processing."""
    frame = preprocess_image(frame)
    
    if config.device_type == 'raspberry-pi':
        # TFLite inference
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        interpreter.set_tensor(input_details[0]['index'], np.expand_dims(frame, 0))
        interpreter.invoke()
        results = interpreter.get_tensor(output_details[0]['index'])
    else:
        # PyTorch inference
        results = model(frame, conf=CONF_THRESHOLD)[0]
    
    return results

def run_inference(video_path):
    """Memory-efficient video processing."""
    cap = cv2.VideoCapture(video_path)
    box_annotator = BoxAnnotator()
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Monitor resources
            metrics = monitor.collect_metrics()
            warnings = monitor.check_thresholds(metrics)
            
            if warnings:
                print("Resource warnings:", warnings)
                continue  # Skip frame if resources are constrained
            
            # Process frame
            results = process_frame(frame)
            
            # Clear unused variables to manage memory
            del frame
            
    finally:
        cap.release()
        # Clean up resources
        if 'interpreter' in locals():
            del interpreter
